# Recurrent Neural Networks
- Weights shared for each "word cell" (recurring)
![image.png](img/rnn.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/TyJuk/recurrent-neural-networks)
<br>
<br>

# Typical RNN Tasks
- 1:n - e.g. give a word and get a sentence
- n:1 - e.g. give a sentence and figure out if it's offensive or not (binary classification)
- n:n - e.g. translate a sentence into another language
<br>
<br>

# Why?
![image.png](img/rnn_q_matrix.png)
<br>
<br>

# Simple RNN
![image.png](img/simple_rnn.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/eaLt6/math-in-simple-rnns)
<br>
<br>

# Formulas
![image-2.png](img/rnn_math.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/eaLt6/math-in-simple-rnns)

<br>
<br>
<img src="img/rnn_element_wise.png" align="left"/>

This funny circle is standing for "*a binary operation that takes two matrices of the same dimensions and produces another matrix of the same dimension as the operands, where each element i, j is the product of elements i, j of the original two matrices. It is to be distinguished from the more common matrix product.*" [(Hadamard product)](https://en.wikipedia.org/wiki/Hadamard_product_(matrices)) Also see: [How to do in Python](https://stackoverflow.com/questions/40034993/how-to-get-element-wise-matrix-multiplication-hadamard-product-in-numpy)

<br>
<br>

# What are we training?
![image.png](img/rnn_what_to_train.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/eaLt6/math-in-simple-rnns)

<br>
<br>

# Calculate Hidden State Activation `h` in Python
*From h_t_prev to h_t*

In [1]:
import numpy as np

In [2]:
w_hh = np.random.standard_normal((3,2))
w_hx = np.random.standard_normal((3,3))
h_t_prev = np.random.standard_normal((2,1))
x_t = np.random.standard_normal((3,1))

In [3]:
print("w_hh",w_hh,"w_hx",w_hx,"h_t_prev",h_t_prev,"x_t",x_t, sep="\n\n")

w_hh

[[ 0.12967326 -0.27805933]
 [-0.68666299  0.05393412]
 [ 0.40413422 -2.06988829]]

w_hx

[[-1.03164011 -0.8504492  -0.32841076]
 [-0.42133041 -0.77793715 -0.16233236]
 [ 1.57972247 -0.69032188  0.02990385]]

h_t_prev

[[ 0.66448566]
 [-1.23570622]]

x_t

[[0.44797211]
 [1.20045448]
 [0.37405119]]


In [4]:
def sigmoid(x):
     return 1 / (1 + np.exp(-x))
    
bias = np.random.standard_normal((x_t.shape[0],1))
bias

array([[-0.7419451 ],
       [ 0.0439067 ],
       [ 1.09225621]])

In [5]:
h_t = sigmoid(np.matmul(w_hh, h_t_prev) + np.matmul(w_hx, x_t) + bias)
print(h_t)

A = h_t

[[0.12807433]
 [0.15944535]
 [0.97830477]]


In [6]:
# Another way
h_t = sigmoid(np.matmul(np.hstack((w_hh, w_hx)), np.vstack((h_t_prev, x_t))) + bias)
print(h_t)

B = h_t

[[0.12807433]
 [0.15944535]
 [0.97830477]]


In [7]:
np.allclose(A,B)

True

In [8]:
# Another way, too
sigmoid(np.hstack((w_hh, w_hx)) @ np.vstack((h_t_prev, x_t)) + bias)

array([[0.12807433],
       [0.15944535],
       [0.97830477]])

In [9]:
# Another way, too (this one is sexy)
sigmoid(w_hh @ h_t_prev + w_hx @ x_t + bias)

array([[0.12807433],
       [0.15944535],
       [0.97830477]])

In [35]:
#np.concatenate([h_t_prev, x_t])
#w_hh.shape
#sigmoid(np.dot(w_hh, np.concatenate([h_t_prev, x_t])) + bias)

array([[0.19276986, 0.22261268, 0.31235083],
       [0.70758389, 0.65986792, 0.52531488],
       [0.03087153, 0.09515606, 0.68606918]])

In [10]:
%timeit sigmoid(np.matmul(w_hh, h_t_prev) + np.matmul(w_hx, x_t) + bias)

6.19 µs ± 242 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%timeit sigmoid(np.matmul(np.hstack((w_hh, w_hx)), np.vstack((h_t_prev, x_t))) + bias)

13.9 µs ± 693 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
%timeit sigmoid(np.hstack((w_hh, w_hx)) @ np.vstack((h_t_prev, x_t)) + bias)

14 µs ± 585 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [13]:
%timeit sigmoid(w_hh @ h_t_prev + w_hx @ x_t + bias)

5.92 µs ± 465 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Lets use `@` for element-wise operations. It's easier to remember, shorter and faster.

# Costs
For one example costs can be calculated like this.

<img src="img/rnn_costs_single.png" align="left"/>

<br>





If several time steps *T* are involved, we're building average costs.

<img src="img/rnn_costs_all.png" align="left"/>

[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/KBmVE/cost-function-for-rnns)

# Scan functions
- Abstract RNNs for fast computation
- Needed for GPU usage & parrallel computing

![image.png](img/rnn_scan_functions.png)

[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/rhso8/implementation-note)

An evaluation metric used in this course is perplexity, which seems not to fit with [speech recognition tasks](https://www.researchgate.net/post/What-are-the-performance-measures-in-Speech-recognition) (?).

Some more intuitive explanations towards perplexity:
- https://towardsdatascience.com/evaluation-of-language-models-through-perplexity-and-shannon-visualization-method-9148fbe10bd0
- https://www.cs.cmu.edu/~roni/papers/eval-metrics-bntuw-9802.pdf
- https://www.quora.com/How-does-perplexity-function-in-natural-language-processing?share=1

# Gated Recurrent Units (GRU)

- In simple RNNs hidden state *h* gets updated from unit to unit
- Problematic for long sequences (information loss each step -> vanishing gradients)
- GRUs have additional formulas (gates) to compute to keep relevant information available through all states

## Vanilla & GRU
![simple_rnn_vs_gru.png](img/simple_rnn_vs_gru.png)
[Source](https://www.coursera.org/learn/sequence-models-in-nlp/supplement/t5L3H/gated-recurrent-units)

The symbol $\Gamma$ looking like a 'T' without its left 'arm' is called *gamma* (Greek). It is used for the equations of 'gates' taking care of updates ($\Gamma_u$) & relevance ($\Gamma_r$).

# Testing simple RNNs & GRUs & Scan functions

Following [this](https://www.coursera.org/learn/sequence-models-in-nlp/ungradedLab/jJn3o/vanilla-rnns-grus-and-the-scan-function) lab... Although, I'm wondering why we're calculating hidden states now in a different way than we did it before.

In [39]:
from numpy import random

random.seed(10)                 

emb = 128                       # Embedding size
T = 256                         # Number of variables in the sequences
h_dim = 16                      # Hidden state dimension
h_0 = np.zeros((h_dim, 1))      # Initial hidden state

w1 = random.standard_normal((h_dim, emb+h_dim))
w2 = random.standard_normal((h_dim, emb+h_dim))
w3 = random.standard_normal((h_dim, emb+h_dim))

b1 = random.standard_normal((h_dim, 1))
b2 = random.standard_normal((h_dim, 1))
b3 = random.standard_normal((h_dim, 1))

X = random.standard_normal((T, emb, 1))

weights = [w1, w2, w3, b1, b2, b3]

In [49]:
def forward_V_RNN(inputs, weights):
    x, h_t = inputs
    wh, _, _, bh, _, _ = weights

    # Returning new hidden state only
    h_t = sigmoid(np.dot(wh, np.concatenate([h_t, x])) + bh)
    
    return h_t, h_t

In [50]:
def forward_GRU(inputs, weights): # Forward propagation for a single GRU cell
    x, h_t = inputs
    wu, wr, wc, bu, br, bc = weights

    # Update gate
    u = sigmoid(np.dot(wu, np.concatenate([h_t, x])) + bu)
    
    # Relevance gate
    r = sigmoid(np.dot(wr, np.concatenate([h_t, x])) + br)

    
    # Candidate hidden state 
    c = np.dot(wc, np.concatenate([r * h_t, x])) + bc
    c = np.tanh(c)
    
    # Returning new hidden state only
    h_t = u * c + (1 - u) * h_t
    
    return h_t, h_t

In [44]:
def scan(fn, elems, weights, h_0=None):
    h_t = h_0
    ys = []
    
    for x in elems:
        y, h_t = fn([x, h_t], weights)
        ys.append(y)
    
    return ys, h_t

In [47]:
%timeit scan(forward_V_RNN, X, weights, h_0)

1.87 ms ± 47.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:
%timeit scan(forward_GRU, X, weights, h_0)

5.41 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
